In [1]:
!pip install transformers torch nltk flask

   ---------------------------------------- 0.0/1.5 MB ? eta -:--:--
   ------------- -------------------------- 0.5/1.5 MB 10.5 MB/s eta 0:00:01
   ----------------------------- ---------- 1.1/1.5 MB 14.2 MB/s eta 0:00:01
   ---------------------------------------- 1.5/1.5 MB 12.0 MB/s eta 0:00:00
   ---------------------------------------- 0.0/101.7 kB ? eta -:--:--
   ---------------------------------------- 101.7/101.7 kB ? eta 0:00:00



[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [10]:
import pandas as pd
data = pd.read_csv('Intents.csv')
print(data.head())

                                      User Input               Intent  \
0            "Book a flight to Manila on Monday"       Flight Booking   
1          "I want to cancel my flight to Davao"  Flight Cancellation   
2  "Can I change my flight from Cebu to Manila?"        Change Flight   
3                 "How much baggage can I take?"      Baggage Inquiry   
4                        "Is my flight on time?"  Check Flight Status   

                                        Entities  
0  "{'destination': 'Manila', 'date': 'Monday'}"  
1                     "{'destination': 'Davao'}"  
2  "{'origin': 'Cebu', 'destination': 'Manila'}"  
3                                           "{}"  
4                                           "{}"  


In [23]:
import pandas as pd
import torch
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer, BertForSequenceClassification, pipeline, AdamW
from torch.utils.data import Dataset, DataLoader

In [24]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [25]:
ner_model = pipeline("ner", model="dbmdz/bert-large-cased-finetuned-conll03-english")

Some weights of the model checkpoint at dbmdz/bert-large-cased-finetuned-conll03-english were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [26]:
data = pd.read_csv('Intents.csv')
print(data.head())

                                      User Input               Intent  \
0            "Book a flight to Manila on Monday"       Flight Booking   
1          "I want to cancel my flight to Davao"  Flight Cancellation   
2  "Can I change my flight from Cebu to Manila?"        Change Flight   
3                 "How much baggage can I take?"      Baggage Inquiry   
4                        "Is my flight on time?"  Check Flight Status   

                                        Entities  
0  "{'destination': 'Manila', 'date': 'Monday'}"  
1                     "{'destination': 'Davao'}"  
2  "{'origin': 'Cebu', 'destination': 'Manila'}"  
3                                           "{}"  
4                                           "{}"  


In [27]:
from transformers import pipeline


ner = pipeline("ner", model="dbmdz/bert-large-cased-finetuned-conll03-english")


user_input = "Book a flight to Paris for tomorrow"


entities = ner(user_input)
print("Entities:", entities)

Some weights of the model checkpoint at dbmdz/bert-large-cased-finetuned-conll03-english were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Entities: [{'entity': 'I-LOC', 'score': 0.9996762, 'index': 5, 'word': 'Paris', 'start': 17, 'end': 22}]


In [28]:
intent_labels = {intent: idx for idx, intent in enumerate(data['Intent'].unique())}
data['Label'] = data['Intent'].map(intent_labels)

In [29]:
train_data, test_data = train_test_split(data, test_size=0.2, random_state=42)

In [30]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [33]:
def tokenize_data(df):
    return tokenizer(list(df['User Input']), padding=True, truncation=True, return_tensors='pt')

train_encodings = tokenize_data(train_data)
test_encodings = tokenize_data(test_data)

In [34]:
class IntentDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

In [35]:
train_dataset = IntentDataset(train_encodings, train_data['Label'].tolist())
test_dataset = IntentDataset(test_encodings, test_data['Label'].tolist())

In [36]:
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=len(intent_labels))

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [37]:
optimizer = AdamW(model.parameters(), lr=5e-5)

c:\Users\Jhaide\Downloads\ChatBot Jotter-O\.env\Lib\site-packages\transformers\optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [38]:
model.train()
for epoch in range(3):  # Number of training epochs
    for batch in DataLoader(train_dataset, batch_size=8, shuffle=True):
        optimizer.zero_grad()
        outputs = model(**batch)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
    print(f'Epoch {epoch + 1} finished.')

C:\Users\Jhaide\AppData\Local\Temp\ipykernel_19836\2198959674.py:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}


Epoch 1 finished.
Epoch 2 finished.
Epoch 3 finished.


In [39]:
ner_pipeline = pipeline("ner", model="dbmdz/bert-large-cased-finetuned-conll03-english")

Some weights of the model checkpoint at dbmdz/bert-large-cased-finetuned-conll03-english were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [40]:
user_input = "I need to fly to Singapore from the Philippines next week"

In [44]:
inputs = tokenizer(user_input, return_tensors='pt', padding=True, truncation=True)
with torch.no_grad():
    outputs = model(**inputs)
    predicted_label = torch.argmax(outputs.logits, dim=1).item()
    predicted_intent = [intent for intent, idx in intent_labels.items() if idx == predicted_label][0]


entities = ner_pipeline(user_input)


print("Entities:", entities)

Entities: [{'entity': 'I-LOC', 'score': 0.99986756, 'index': 6, 'word': 'Singapore', 'start': 17, 'end': 26}, {'entity': 'I-LOC', 'score': 0.99985933, 'index': 9, 'word': 'Philippines', 'start': 36, 'end': 47}]


In [50]:
feedback = []

def get_user_feedback():
    rating = input("On a scale of 1 to 5, how would you rate my response? ")
    try:
        rating = int(rating)
        if 1 <= rating <= 5:
            feedback.append(rating)
            print("Thank you for your feedback!")
        else:
            print("Please provide a rating between 1 and 5.")
    except ValueError:
        print("Invalid input. Please enter a number between 1 and 5.")

def adjust_response():
    if not feedback:
        return "I have not received any feedback yet."
    avg_feedback = sum(feedback) / len(feedback)
    if avg_feedback < 3:
        return "Sorry, I’ll improve my response next time."
    else:
        return "Thank you for the feedback! I'm glad you found my response helpful."

user_input = "Can you help me book a flight to Manila?"

response = "You can book a flight to Manila through our website or mobile app."

print("Chatbot:", response)


print("User Feedback: ", get_user_feedback())


adjusted_response = adjust_response()
print("Adjusted Response: ", adjusted_response)

Chatbot: You can book a flight to Manila through our website or mobile app.
Thank you for your feedback!
User Feedback:  None
Adjusted Response:  Thank you for the feedback! I'm glad you found my response helpful.
